In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\' + filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_176340/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_176340/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_176340/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_176340/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
703,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
704,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.52,3.435


Now filter the dataframe

In [6]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["LTCUSDT","ONEUSDT","COTIUSDT","ATOMUSDT"]
big_data['adgt_%'] = big_data['adgl_%']+big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1 )
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%,adgt_%
113,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9660,True,True,0.0921,0.0306,100.0,100.0,0.9829,20.42,6.290,0.1227
114,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9660,True,True,0.0921,0.0306,100.0,100.0,0.9829,20.42,6.290,0.1227
115,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9660,True,True,0.0921,0.0306,100.0,100.0,0.9829,20.42,6.290,0.1227
116,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9543,True,True,0.1280,0.0444,100.0,100.0,0.8517,29.46,7.192,0.1724
117,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9509,True,True,0.1060,0.0710,100.0,100.0,0.8820,24.29,13.860,0.1770
118,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9543,True,True,0.1280,0.0444,100.0,100.0,0.8517,29.46,7.192,0.1724
119,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9478,True,True,0.1880,0.0768,100.0,100.0,0.8554,45.26,15.100,0.2648
120,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9509,True,True,0.1060,0.0710,100.0,100.0,0.8820,24.29,13.860,0.1770
121,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9522,True,True,0.1120,0.0546,100.0,100.0,0.8807,25.79,9.824,0.1666
122,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,5.5,ATOMUSDT,0.9478,True,True,0.1880,0.0768,100.0,100.0,0.8554,45.26,15.100,0.2648


In [7]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_176340/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
113,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,ATOMUSDT,0.1227,0.0921,0.0306,100.0,100.0,0.9660,0.9829
116,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,ATOMUSDT,0.1724,0.1280,0.0444,100.0,100.0,0.9543,0.8517
117,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,ATOMUSDT,0.1770,0.1060,0.0710,100.0,100.0,0.9509,0.8820
119,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,ATOMUSDT,0.2648,0.1880,0.0768,100.0,100.0,0.9478,0.8554
121,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,ATOMUSDT,0.1666,0.1120,0.0546,100.0,100.0,0.9522,0.8807
123,C:\AgodaGit\passivbot\backtests/binance\ATOMUS...,ATOMUSDT,0.1808,0.1150,0.0658,100.0,100.0,0.9525,0.8916
209,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,COTIUSDT,0.1639,0.0985,0.0654,100.0,100.0,0.9508,0.8319
212,C:\AgodaGit\passivbot\backtests/binance\COTIUS...,COTIUSDT,0.2060,0.1430,0.0630,100.0,100.0,0.9185,0.8065
478,C:\AgodaGit\passivbot\backtests/binance\LTCUSD...,LTCUSDT,0.1011,0.0505,0.0506,100.0,100.0,0.9553,0.9273
546,C:\AgodaGit\passivbot\backtests/binance\ONEUSD...,ONEUSDT,0.1745,0.1030,0.0715,100.0,100.0,0.9391,0.9622
